In [1]:
import pandas as pd
import psycopg2

In [2]:
# Connect to postgres
dbname = "weather"
user = "root"
password = "root"
host = "localhost"
port = "5401"

In [3]:
file_path = '../../dataset-Climate/Population.csv'
year_range = range(1960, 2013+1)
country_col_name = 'Country Code'
population_df = pd.read_csv(file_path)

In [4]:
# Establish a connection to the PostgreSQL database
try:
    connection = psycopg2.connect(
        dbname=dbname,
        user=user,
        password=password,
        host=host,
        port=port
    )

    # Create a cursor object to interact with the database
    cursor = connection.cursor()

    # Read the Population.csv file
    

    insert_count = 0
    for index, row in population_df.iterrows():
        country_code = row[country_col_name]
        # Extract the years and corresponding amounts
        for year in year_range:
            # print(country_code, year)
            amount = row[str(year)]
            # Handle NaN values and set amount to NULL
            amount_sql = 'NULL' if pd.isna(amount) or pd.isnull(amount) else str(amount)

            # Check if the data already exists
            check_query = f"""
            SELECT 1
            FROM population
            WHERE year = {year} AND country_code = '{country_code}'
            LIMIT 1;
            """
            cursor.execute(check_query)
            data_exists = cursor.fetchone()

            if not data_exists:
                # Data does not exist, proceed with the insert
                insert_query = f"""
                INSERT INTO population (year, country_code, amount)
                VALUES ({year}, '{country_code}', {amount_sql});
                """
                cursor.execute(insert_query)
                insert_count += 1  # Increment the insert count

        

    # Commit the changes and close the cursor and connection
    connection.commit()
    cursor.close()
    connection.close()

    print(f"Data inserted successfully! Total inserts: {insert_count}")


except psycopg2.Error as e:
    print(f"Error connecting to PostgreSQL: {e}")


Data inserted successfully! Total inserts: 11178
